### Experiment: Performance

**Question**: What is the performance on the first SAT dataset for each of the models?

**Hypothesis**: 

**Result**:

In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT1_STAGES_SPEED
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from mne.io import read_info
import gc
import json
from hmpai.training import k_fold_cross_validate_sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [2]:
set_global_seed(42)
data_path = Path("../data/sat2/split_stage_data_100hz.nc")

dataset = xr.load_dataset(data_path)
normalization_fn = norm_min1_to_1

In [3]:
def test_performance(model_fn, model_kwargs, data, additional_train_kwargs=None):
    print(f"Testing model: {model_fn.__name__}")
    train_kwargs = {
        "logs_path": Path("../logs/exp_performance/"),
        "additional_info": {
            "model_fn": model_fn.__name__,
            "model_kwargs": str(model_kwargs),
        },
        "additional_name": f"model_fn-{model_fn.__name__}",
        "labels": SAT1_STAGES_ACCURACY,
    }
    if additional_train_kwargs is not None:
        train_kwargs.update(additional_train_kwargs)
    result = k_fold_cross_validate(
        model_fn,
        model_kwargs,
        data,
        k=len(data.participant),
        normalization_fn=normalization_fn,
        train_kwargs=train_kwargs,
    )
    with open(
        train_kwargs["logs_path"] / f"results_{model_fn.__name__}.json", "w"
    ) as f:
        json.dump(result, f, indent=4)
    print_results(result)


def test_performance_sklearn(model, data):
    print(f"Testing model: {model.__class__.__name__}")
    logs_path = Path("../logs/exp_performance/")
    result = k_fold_cross_validate_sklearn(model, data, k=len(data.participant), normalization_fn=normalization_fn)
    with open(logs_path / f"results_{model.__class__.__name__}.json", "w") as f:
        json.dump(result, f, indent=4)
    print_results(result)

### Baseline (Random Forest)

In [4]:
model = RandomForestClassifier(random_state=42)
test_performance_sklearn(model, dataset)

Testing model: RandomForestClassifier
Fold 1: test fold: ['S1']
Fold 1: Accuracy: 0.7074653755207747
Fold 1: F1-Score: 0.6354298677552853
Fold 2: test fold: ['S10']
Fold 2: Accuracy: 0.6848898216159496
Fold 2: F1-Score: 0.6408738279902068
Fold 3: test fold: ['S18']
Fold 3: Accuracy: 0.7126448362720403
Fold 3: F1-Score: 0.645462690886676
Fold 4: test fold: ['S15']
Fold 4: Accuracy: 0.6756756756756757
Fold 4: F1-Score: 0.6110255401864892
Fold 5: test fold: ['S12']
Fold 5: Accuracy: 0.6776230083174667
Fold 5: F1-Score: 0.6234374796541595
Fold 6: test fold: ['S5']
Fold 6: Accuracy: 0.642842680704596
Fold 6: F1-Score: 0.5819697392951604
Fold 7: test fold: ['S8']
Fold 7: Accuracy: 0.6063477565932184
Fold 7: F1-Score: 0.5368307466534431
Fold 8: test fold: ['S7']
Fold 8: Accuracy: 0.6731958762886598
Fold 8: F1-Score: 0.5952201424786348
Fold 9: test fold: ['S3']
Fold 9: Accuracy: 0.7012987012987013
Fold 9: F1-Score: 0.6093160833881559
Fold 10: test fold: ['S11']
Fold 10: Accuracy: 0.69994308480

### Baseline (SVM)

In [5]:
model = SVC(kernel="rbf", random_state=42)
test_performance_sklearn(model, dataset)

Testing model: SVC
Fold 1: test fold: ['S1']
Fold 1: Accuracy: 0.7704087377547574
Fold 1: F1-Score: 0.7033334838580079
Fold 2: test fold: ['S10']
Fold 2: Accuracy: 0.7424973767051417
Fold 2: F1-Score: 0.7142053988474251
Fold 3: test fold: ['S18']
Fold 3: Accuracy: 0.7727959697732998
Fold 3: F1-Score: 0.736386854243346
Fold 4: test fold: ['S15']
Fold 4: Accuracy: 0.7604846225535881
Fold 4: F1-Score: 0.7203313891465007
Fold 5: test fold: ['S12']
Fold 5: Accuracy: 0.7582924140695461
Fold 5: F1-Score: 0.7200403768949604
Fold 6: test fold: ['S5']
Fold 6: Accuracy: 0.7584531281635959
Fold 6: F1-Score: 0.7173881097515707
Fold 7: test fold: ['S8']
Fold 7: Accuracy: 0.711725082772006
Fold 7: F1-Score: 0.6270906560823198
Fold 8: test fold: ['S7']
Fold 8: Accuracy: 0.7542955326460481
Fold 8: F1-Score: 0.6913970624348931
Fold 9: test fold: ['S3']
Fold 9: Accuracy: 0.7692467532467533
Fold 9: F1-Score: 0.6835332406327141
Fold 10: test fold: ['S11']
Fold 10: Accuracy: 0.7633466135458168
Fold 10: F1-S

### CNN

In [6]:
test_performance(
    SAT1Base,
    {"n_classes": len(dataset.labels)},
    dataset,
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: SAT1Base
Fold 1: test fold: ['S1']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

...

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

Fold 18: Accuracy: 0.9067717996289425
Fold 18: F1-Score: 0.9003265407884236
Test set 0
Accuracies
[0.905753856547686, 0.8977964323189926, 0.8968261964735517, 0.9005902454178316, 0.8753382102415072, 0.9070662077343592, 0.9193971914602124, 0.8592210767468499, 0.8797922077922078, 0.8973249857712009, 0.8289357180803967, 0.9313212195873114, 0.9326778368616434, 0.9337264453275221, 0.942449898188833, 0.8983896575187117, 0.9183027923211169, 0.9067717996289425]
F1-Scores
[0.8981685651385167, 0.8923185943918513, 0.8878501872280811, 0.8935748839546986, 0.869174692264524, 0.8984111864129074, 0.9192980940556004, 0.848172885635746, 0.8688676815645984, 0.8910349155644315, 0.8214508643666345, 0.9326248558990841, 0.9303475093189123, 0.9256783294071372, 0.9388306796347525, 0.8977784800666242, 0.9099516151368684, 0.9003265407884236]
Average Accuracy: 0.9017601098899376, std: 0.02741384404530056
Average F1-Score: 0.8957700311571885, std: 0.02926561202425858


### GRU

In [7]:
test_performance(
    SAT1GRU,
    {
        "n_classes": len(dataset.labels),
        "n_samples": len(dataset.samples),
        "n_channels": len(dataset.channels),
    },
    dataset,
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: SAT1GRU
Fold 1: test fold: ['S1']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

...

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

Fold 18: Accuracy: 0.9173237476808905
Fold 18: F1-Score: 0.9121588910821762
Test set 0
Accuracies
[0.9148744510753294, 0.9061909758656873, 0.9113350125944585, 0.9105312208760484, 0.8936767211143402, 0.9190119457380036, 0.9293298321726224, 0.8936998854524628, 0.8909090909090909, 0.8890153671030165, 0.8489463431910749, 0.9272148650035931, 0.9410053175479081, 0.935521087040123, 0.9479155503161505, 0.8983896575187117, 0.9297556719022687, 0.9173237476808905]
F1-Scores
[0.9080098864843796, 0.9032827236391396, 0.9081329981197254, 0.9049579432954704, 0.8897391435406693, 0.9121444017969651, 0.9301494130711163, 0.8864947073995231, 0.8829349997085295, 0.883263705782638, 0.8498131272521373, 0.9290334805365946, 0.9416253484275234, 0.9282479533330443, 0.946755806411474, 0.900582774462557, 0.9237895011258501, 0.9121588910821762]
Average Accuracy: 0.9113692635056546, std: 0.022914421620231624
Average F1-Score: 0.907839822526084, std: 0.023294379311131538


### Transformer

In [8]:
test_performance(
    TransformerModel,
    {
        "n_features": len(dataset.channels),
        "n_heads": 10,
        "ff_dim": 512,
        "n_layers": 6,
        "n_samples": len(dataset.samples),
        "n_classes": len(dataset.labels),
    },
    dataset,
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: TransformerModel
Fold 1: test fold: ['S1']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

...

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

Fold 18: Accuracy: 0.8697820037105751
Fold 18: F1-Score: 0.8613878907987917
Test set 0
Accuracies
[0.8884134669519198, 0.8927597061909759, 0.876977329974811, 0.8691104898001449, 0.7302334903296923, 0.9091921441587366, 0.8880009133462724, 0.8560137457044673, 0.8725194805194805, 0.8471257825839499, 0.8129980520630423, 0.8584334257263114, 0.9183304906190428, 0.8116908088706576, 0.9187654056371236, 0.838398729870719, 0.8921247818499127, 0.8697820037105751]
F1-Scores
[0.8779717374550234, 0.8886151426284702, 0.8683334244228158, 0.8621524080433153, 0.6785377656335452, 0.9023173427590117, 0.8887924562893408, 0.843366647129776, 0.8598218997544642, 0.8397036283872021, 0.8084189683274616, 0.8567404289846209, 0.9168101734348164, 0.7068200058654404, 0.9159378159134732, 0.8351395348433537, 0.8810576931801046, 0.8613878907987917]
Average Accuracy: 0.8639372359948797, std: 0.04431572204212684
Average F1-Score: 0.8495513868806126, std: 0.06193315498476651
